#Proyecto Chatbot Customer Service

In [1]:
!pip install transformers
!pip install numpy
!pip install pandas
!pip install datasets
!pip install sklearn
!pip install torch
!pip install logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post11-py3-none-any.whl size=2959 sha256=4f299445a227d8a5762cc64e646f11b8aeead0786963cd116a8eff3801d7f26d
  Stored in directory: /root/.cache/pip/wheels/aa/9c/60/f67813603a52fc35057868f1aba0003cc75b72583dcaa2c341
Successfully built sklearn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 1.8 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generat

In [2]:
!pip install accelerate -U
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.6 MB/s eta 0:00:00


In [3]:
!pip show accelerate

Name: accelerate
Version: 0.24.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import TrainingArguments
import torch

Load dataset from a csv or directly from hugginface

In [6]:
# Load your dataset
dataset = pd.read_csv('converted.csv')
dataset.head()


,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for contacting BrownBox custo...


In [8]:
from sklearn.model_selection import train_test_split

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("Kaludi/Customer-Support-Assistant-V2")

# Extract the 'conversation' column
conversations = dataset['conversation']

# Basic preprocessing (optional based on your requirement)
conversations = conversations.str.lower()
#conversations = conversations.str.replace(r'[^\w\s]', '')
conversations.dropna(inplace=True)

# Split the preprocessed conversations into training and test sets
train_conversations, test_conversations = train_test_split(conversations, test_size=0.2, random_state=42)



# Tokenize the training data
tokenized_train_data = tokenizer(
    train_conversations.tolist(),
    padding="max_length",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

# Tokenize the test data
tokenized_test_data = tokenizer(
    test_conversations.tolist(),
    padding="max_length",
    truncation=True,
    max_length=512,
    return_tensors="pt"
)


In [25]:
conversations.head()

0    agent: thank you for calling brownbox customer...
1    agent: thank you for calling brownbox customer...
2    agent: thank you for calling brownbox customer...
3    customer: hi, i am facing an issue while loggi...
4    agent: thank you for contacting brownbox custo...
Name: conversation, dtype: object

Función para separar la conversación entre inputs y respuestas target

In [26]:
def parse_conversation(conversations):
    customer_inputs = []
    agent_responses = []
    for conversation in conversations:
        turns = conversation.split('\n')
        for i in range(len(turns)):
            if turns[i].startswith("customer:"):
                # Extract customer message, removing the prefix and leading/trailing whitespace
                customer_msg = turns[i].split("customer: ", 1)[1].strip()
                # Initialize agent message as empty
                agent_msg = ""
                # Look ahead for the next agent message
                for j in range(i+1, len(turns)):
                    if turns[j].startswith("agent:"):
                        agent_msg = turns[j].split("agent: ", 1)[1].strip()
                        break  # Stop at the first agent response after the customer message
                customer_inputs.append(customer_msg)
                agent_responses.append(agent_msg)
    return customer_inputs, agent_responses


In [27]:
train_customer_inputs, train_agent_responses = parse_conversation(train_conversations)
test_customer_inputs, test_agent_responses = parse_conversation(test_conversations)


In [28]:
train_customer_inputs[:5]


["hi sarah, i'm calling to change the address for the item pick-up of my pram/stroller.",
 'my order number is 789012 and i purchased it on the 15th of this month.',
 'yes, the new address is 123 main street, anytown, usa.',
 "no, it's not within the same zip code.",
 "this is ridiculous. i don't want to cancel the order. i need the pram/stroller, and i cannot wait for a new one to arrive."]

In [30]:
train_agent_responses[:10]

["i'm sorry to hear that you need to change the address. i'll be happy to assist you with that. can you please provide me with your order number and the date of purchase?",
 'thank you for the information. i understand that you want to change the address for the item pick-up. can you please confirm the new address?',
 "thank you for providing the new address. let me check if it's possible to change the address for the item pick-up. please hold on for a moment.",
 "i'm sorry, but we cannot change the address for the item pick-up if it's not within the same zip code. you can cancel the order and place a new one with the correct address.",
 'i completely understand your situation. let me check if there are any other options available. please hold on for a moment.',
 "you're welcome. let me make the necessary arrangements. please hold on for a moment.",
 "you're welcome. is there anything else i can help you with?",
 "you're welcome. have a great day!",
 "hello, i'm sorry to hear that you'

In [31]:
# Tokenize training data
tokenized_train_inputs = tokenizer(train_customer_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_train_labels = tokenizer(train_agent_responses, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Tokenize test data
tokenized_test_inputs = tokenizer(test_customer_inputs, padding=True, truncation=True, max_length=512, return_tensors="pt")
tokenized_test_labels = tokenizer(test_agent_responses, padding=True, truncation=True, max_length=512, return_tensors="pt")


In [32]:
class ConversationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [33]:
#Datasets Instances
train_dataset = ConversationDataset(tokenized_train_inputs, tokenized_train_labels)
test_dataset = ConversationDataset(tokenized_test_inputs, tokenized_test_labels)


In [34]:
train_dataset[0]

{'input_ids': tensor([    0,  3592,   579, 36000,     6,   939,   437,  1765,     7,   464,
             5,  1100,    13,     5,  6880,  1339,    12,   658,     9,   127,
          3349,   424,    73,   620, 32121,     4,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([   0,  118,  437, 6661,    7, 1798,   14,   47,  240,    7,  464,    5,
         1100,    4,  939,  581,   28, 1372,    7, 3991,   47,   19,   14,    4,
           64

In [35]:
test_dataset[0]

{'input_ids': tensor([    0,  3592,   579, 36000,     6,   939,   524,   667,     7,   120,
            11,  2842,    19,     5, 37171,   544,  3696,  3059,    19,     5,
         15689,     9,    10,  3349,   424,    73,   620, 32121,    14,   939,
            33,  2162,    31,  6219,  8304,     4,    64,    47,   244,   162,
            19,    14,   116,     2,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([    0, 41404,     6,   939,  1017,    28,  1372,     7,  3991,    47,
            19,    14,     4,   189,   939,    33,   110,   645,   346,     6,
          2540,   116,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,    

Load Pretrained Model

In [36]:
model = AutoModelForSeq2SeqLM.from_pretrained("Kaludi/Customer-Support-Assistant-V2", from_tf=True)


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BartForConditionalGeneration.

Some weights of BartForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

In [37]:
import logging

logging.basicConfig(level=logging.INFO)


In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',              # Directory for model outputs
    num_train_epochs=3,                  # Total number of training epochs
    per_device_train_batch_size=8,       # Batch size per device during training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    warmup_steps=500,                    # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                   # Weight decay if applied
    logging_dir='./logs',                # Directory for storing logs
    logging_steps=50,                    # Log every X updates steps
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",               # Save model checkpoint at the end of each epoch
    load_best_model_at_end=True,         # Load the best model at the end of training
    max_steps=1000,                      # Total number of training steps to perform
    metric_for_best_model='loss',        # Use loss to determine the best model
)


In [39]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Add this function to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
# Save the model
model.save_pretrained('your_model_directory')

# Load the model for inference
model = AutoModelForSeq2SeqLM.from_pretrained('your_model_directory')
tokenizer = AutoTokenizer.from_pretrained("Kaludi/Customer-Support-Assistant-V2")

In [ ]:
def generate_response(input_text):
    # Tokenize the input text - convert text to a format the model understands
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate a response using the model
    output = model.generate(input_ids)

    # Decode the generated response back into human-readable text
    response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response



In [ ]:
#Conversación de Prueba

user_input = "How can I track my order?"
response = generate_response(user_input)
print("Chatbot:", response)


In [ ]:
#Interacción Continua:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    response = generate_response(user_input)
    print("Chatbot:", response)
